**Most of a program's run time is spent inside loops**

Mostly function calls inside loops.

**The act of calling these functions itself better be fast!**

### But it doesn't look like it should be fast!

To execute `foo(x,y)`, Julia chooses the **method for foo that is _most_ specific to types of x, and y**.

**Looking up this method can be expensive!** There are hundreds of methods for `+(x,y)` picking the most specific one can involve a lot of head-scratching.

But Julia can do it when **compiling** the code, once and forever!

In [75]:
function sum1(xs)
    s = zero(xs[1])

    for i=1:length(xs)
        s = +(s, xs[i])
    end
    s
end

sum1 (generic function with 1 method)

In [76]:
@code_typed optimize=false sum1([1+2im, 7+9im])

CodeInfo(
│2 1 ─ %1  = (Base.getindex)(xs, 1)::Complex{Int64}
│  │         (s = (Main.zero)(%1))::Complex{Int64}
│4 │   %3  = (Main.length)(xs)::Int64
│  │   %4  = (1:%3)::UnitRange{Int64}
│  │         (#temp# = (Base.iterate)(%4))::Union{Nothing, Tuple{Int64,Int64}}
│  │   %6  = (#temp# === nothing)::Bool
│  │   %7  = (Base.not_int)(%6)::Bool
│  └──       goto #4 if not %7
│  2 ┄ %9  = #temp#::Tuple{Int64,Int64}::Tuple{Int64,Int64}
│  │         (i = (Core.getfield)(%9, 1))::Int64
│  │   %11 = (Core.getfield)(%9, 2)::Int64
│5 │   %12 = s::Complex{Int64}
│  │   %13 = (Base.getindex)(xs, i)::Complex{Int64}
│  │         (s = %12 + %13)::Complex{Int64}
│  │         (#temp# = (Base.iterate)(%4, %11))::Union{Nothing, Tuple{Int64,Int64}}
│  │   %16 = (#temp# === nothing)::Bool
│  │   %17 = (Base.not_int)(%16)::Bool
│  └──       goto #4 if not %17
│  3 ─       goto #2
│7 4 ─       return s
) => Complex{Int64}

In [123]:
@code_typed sum1([1+2im, 7+9im]) # Also gets inlined -- no call to + on Complex anymore!

CodeInfo(
│╻     getindex2 1 ──       (Base.arrayref)(true, xs, 1)::Complex{Int64}
││╻╷╷╷  oftype  │    %2  = %new(Complex{Int64}, 0, 0)::Complex{Int64}
│╻     length4 │    %3  = (Base.arraylen)(xs)::Int64
││╻╷╷╷  Type  │    %4  = (Base.sle_int)(1, %3)::Bool
│││╻     unitrange_last  │          (Base.sub_int)(%3, 1)::Int64
││││    │    %6  = (Base.ifelse)(%4, %3, 0)::Int64
││╻╷╷   isempty  │    %7  = (Base.slt_int)(%6, 1)::Bool
││      └───       goto #3 if not %7
││      2 ──       goto #4
││      3 ──       goto #4
│       4 ┄─ %11 = φ (#2 => true, #3 => false)::Bool
│       │    %12 = φ (#3 => 1)::Int64
│       │    %13 = φ (#3 => 1)::Int64
│       │    %14 = (Base.not_int)(%11)::Bool
│       └───       goto #10 if not %14
│       5 ┄─ %16 = φ (#4 => %2, #9 => %20)::Complex{Int64}
│       │    %17 = φ (#4 => %12, #9 => %26)::Int64
│       │    %18 = φ (#4 => %13, #9 => %27)::Int64
│╻     getindex5 │    %19 = (Base.arrayref)(true, xs, %17)::Complex{Int64}
│       │    %20 = invoke Mai

In [82]:
@noinline add(x,y) = x+y # forcefully avoid inlining

add (generic function with 1 method)

In [83]:
function sum2(xs)
    s = zero(xs[1])

    for i=1:length(xs)
        s = add(s, xs[i])
    end
    s
end

sum2 (generic function with 1 method)

In [84]:
@code_native sum2([1+1im, 2+3im])

	.text
; Function sum2 {
; Location: In[83]:2
	pushq	%rbp
	movq	%rsp, %rbp
	pushq	%r15
	pushq	%r14
	pushq	%r13
	pushq	%r12
	pushq	%rbx
	subq	$56, %rsp
	movq	%rsi, %rbx
; Function getindex; {
; Location: array.jl:731
	movq	8(%rbx), %r14
	testq	%r14, %r14
	je	L217
;}
; Location: In[83]:4
	jle	L191
	movq	%rdi, -48(%rbp)
; Location: In[83]:4
	vxorps	%xmm0, %xmm0, %xmm0
	vmovaps	%xmm0, -80(%rbp)
	movl	$1, %r15d
	xorl	%r12d, %r12d
	leaq	-96(%rbp), %r13
	nopw	%cs:(%rax,%rax)
; Location: In[83]:5
; Function getindex; {
; Location: array.jl:731
L80:
	movq	(%rbx), %rax
	vmovups	(%rax,%r12), %xmm0
	vmovaps	%xmm0, -96(%rbp)
;}
	leaq	-64(%rbp), %rdi
	leaq	-80(%rbp), %rsi
	movq	%r13, %rdx
	movabsq	$julia_add_36169, %rax
	callq	*%rax
; Function iterate; {
; Location: range.jl:575
; Function ==; {
; Location: promotion.jl:425
	cmpq	%r15, %r14
;}}
	je	L180
	vmovaps	-64(%rbp), %xmm0
; Location: In[83]:4
	vmovaps	%xmm0, -80(%rbp)
; Location: In[83]:5
; Function getindex; {
; Location: array.jl:731
	addq	

Let's try to create a function that doesn't allow you to infer its result type

In [128]:
vec1 = [1,2]
@show typeof(vec1)

vec2 = Union{Int, Float64}[1,2.3]
@show typeof(vec2)

vec3 = [1,"hi"]
@show typeof(vec3)
vec[1] # what is the output type of vec[i] for all i?


function bad_add(x,y)
    d=[0, "yes"] # intentionally create a Vector{Any}
    x+y+d[1]
end

typeof(vec1) = Array{Int64,1}
typeof(vec2) = Array{Union{Float64, Int64},1}
typeof(vec3) = Array{Any,1}


bad_add (generic function with 1 method)

In [117]:


function sum3(xs)
    s = zero(xs[1])

    for i=1:length(xs)
        s = bad_add(s, xs[i])
    end
    s
end

sum3 (generic function with 1 method)

In [118]:
@code_typed bad_add(1,2)

CodeInfo(
│     2 1 ── %1  = (Core.tuple)(0)::Tuple{Int64}
│╻     getindex  │    %2  = $(Expr(:foreigncall, :(:jl_alloc_array_1d), Array{Any,1}, svec(Any, Int64), :(:ccall), 2, Array{Any,1}, 1, 1))::Array{Any,1}
││╻╷╷   Colon  │          (Base.ifelse)(true, 1, 0)::Int64
│││╻╷╷   isempty  │    %4  = (Base.slt_int)(1, 1)::Bool
│││     └───       goto #3 if not %4
│││     2 ──       goto #4
│││     3 ──       goto #4
││      4 ┄─ %8  = φ (#2 => true, #3 => false)::Bool
││      │    %9  = φ (#3 => 1)::Int64
││      │    %10 = φ (#3 => 1)::Int64
││      │    %11 = (Base.not_int)(%8)::Bool
││      └───       goto #10 if not %11
││      5 ┄─ %13 = φ (#4 => %9, #9 => %22)::Int64
││      │    %14 = φ (#4 => %10, #9 => %23)::Int64
││╻     getindex  │    %15 = (Base.getfield)(%1, %13, false)::Int64
││╻     setindex!  │          (Base.arrayset)(false, %2, %15, %13)::Array{Any,1}
│││╻     ==  │    %17 = (%14 === 1)::Bool
│││     └───       goto #7 if not %17
│││     6 ──       goto #8
│││╻     +  7

In [119]:
@code_warntype sum3([1,2])

Body::Any
│╻     getindex7  1 ──       (Base.arrayref)(true, xs, 1)
│╻     length9  │    %2  = (Base.arraylen)(xs)::Int64
││╻╷╷╷  Type   │    %3  = (Base.sle_int)(1, %2)::Bool
│││╻     unitrange_last   │          (Base.sub_int)(%2, 1)
││││     │    %5  = (Base.ifelse)(%3, %2, 0)::Int64
││╻╷╷   isempty   │    %6  = (Base.slt_int)(%5, 1)::Bool
││       └───       goto #3 if not %6
││       2 ──       goto #4
││       3 ──       goto #4
│        4 ┄─ %10 = φ (#2 => true, #3 => false)::Bool
│        │    %11 = φ (#3 => 1)::Int64
│        │    %12 = φ (#3 => 1)::Int64
│        │    %13 = (Base.not_int)(%10)::Bool
│        └───       goto #10 if not %13
│        5 ┄─ %15 = φ (#4 => 0, #9 => %19)::Any
│        │    %16 = φ (#4 => %11, #9 => %25)::Int64
│        │    %17 = φ (#4 => %12, #9 => %26)::Int64
│╻     getindex10 │    %18 = (Base.arrayref)(true, xs, %16)::Int64
│        │    %19 = (Main.bad_add)(%15, %18)::Any
││╻     ==   │    %20 = (%17 === %5)::Bool
││       └───       goto #7 if n

### Common pitfall Number 1. use of global variables in loops

In [15]:
xs = rand(10^6)
function sum1_global()
    s = zero(xs[1])
    
    for i=1:length(xs)
        s = +(s, xs[i])
    end
    s
end

sum1_global (generic function with 1 method)

In [24]:
@code_warntype sum1_global()

Body::Any
│    3 1 ─ %1  = (Base.getindex)(Main.xs, 1)::Any
│      │   %2  = (Main.zero)(%1)::Any
│    5 │   %3  = (Main.length)(Main.xs)::Any
│      │   %4  = (isa)(%3, Int64)::Bool
│      └──       goto #3 if not %4
│      2 ─ %6  = π (%3, Int64)
│╻╷╷╷ Colon  │   %7  = (Base.sle_int)(1, %6)::Bool
││╻    Type  │         (Base.sub_int)(%6, 1)
│││┃│   unitrange_last  │   %9  = (Base.sub_int)(1, 1)::Int64
││││   │   %10 = (Base.ifelse)(%7, %6, %9)::Int64
│││    │   %11 = %new(UnitRange{Int64}, 1, %10)::UnitRange{Int64}
│      └──       goto #4
│      3 ─ %13 = (1:%3)::Any
│      └──       goto #4
│      4 ┄ %15 = φ (#2 => %11, #3 => %13)::Any
│      │   %16 = (Base.iterate)(%15)::Any
│      │   %17 = (%16 === nothing)::Bool
│      │   %18 = (Base.not_int)(%17)::Bool
│      └──       goto #7 if not %18
│      5 ┄ %20 = φ (#4 => %2, #6 => %25)::Any
│      │   %21 = φ (#4 => %16, #6 => %26)::Any
│      │   %22 = (Core.getfield)(%21, 1)::Any
│      │   %23 = (Core.getfield)(%21, 2)::Any
│   

### Fixes
- Make the global an argument!
- Make the global a `const`

In [111]:
const arr = rand(10^6)
function sum2_global()
    s = zero(arr[1])
    
    for i=1:length(arr)
        s = s + arr[i]
    end
    s
end

sum2_global (generic function with 1 method)

In [114]:
@time sum1_global()
@time sum2_global()

  0.126528 seconds (5.00 M allocations: 91.537 MiB, 5.34% gc time)
  0.001476 seconds (5 allocations: 176 bytes)


500242.1887876529

**Let's practice propagating type information in our head**

Run through the program and find out the types assigned to each variable over time.

In [27]:
function logint2(n)
    r = 0
    while n > 1
        n = n/2
        r += 1
    end
    r
end

logint2 (generic function with 1 method)

In [32]:
function logint2(n) # assume n::Float64
    r = 0 # Int
    while n > 1 # n :: Float64
        n = n/2 # n :: Float64
        r += 1 # Int
    end
    r # Int
end

logint2 (generic function with 1 method)

In [42]:
@code_typed logint2(10.0)

CodeInfo(
│     1 ─       nothing::Nothing
│   3 2 ┄ %2  = φ (#1 => 0, #3 => %12)::Int64
│     │   %3  = φ (#1 => _2, #3 => %11)::Float64
│╻   >  │   %4  = π (1.0, Float64)
││┃│  <  │   %5  = (Base.lt_float)(%4, %3)::Bool
│││   │   %6  = π (1.0, Float64)
│││╻   ==  │   %7  = (Base.eq_float)(%6, %3)::Bool
│││╻   &  │   %8  = (Base.and_int)(%7, false)::Bool
│││╻   |  │   %9  = (Base.or_int)(%5, %8)::Bool
│     └──       goto #4 if not %9
│╻╷  /4 3 ─ %11 = (Base.div_float)(%3, 2.0)::Float64
│╻   +5 │   %12 = (Base.add_int)(%2, 1)::Int64
│     └──       goto #2
│   7 4 ─       return %2
) => Int64

In [33]:
function logint2(n) # assume n::Int
    r = 0 # Int
    while n > 1 # n :: Union{Int, Float64} (propagated from below)
        n = n/2 # n :: Union{Int, Float64}
        r += 1 # Int
    end
    r # Int
end

logint2 (generic function with 1 method)

In [43]:
@code_typed logint2(10)

CodeInfo(
│       1 ──       nothing::Nothing
│     3 2 ┄─ %2  = φ (#1 => 0, #13 => %41)::Int64
│       │    %3  = φ (#1 => _2, #13 => %40)::Union{Float64, Int64}
│       │    %4  = (isa)(%3, Float64)::Bool
│       └───       goto #4 if not %4
│       3 ── %6  = π (%3, Float64)
│╻╷    >  │    %7  = (Base.sitofp)(Float64, 1)::Float64
││╻     <  │    %8  = (Base.lt_float)(%7, %6)::Bool
│││╻     ==  │    %9  = (Base.eq_float)(%7, %6)::Bool
││││    │    %10 = (Base.eq_float)(%7, 9.22337e18)::Bool
│││╻     unsafe_trunc  │    %11 = (Base.fptosi)(Int64, %7)::Int64
│││╻     <  │    %12 = (Base.slt_int)(1, %11)::Bool
│││╻     |  │    %13 = (Base.or_int)(%10, %12)::Bool
│││╻     &  │    %14 = (Base.and_int)(%9, %13)::Bool
│││╻     |  │    %15 = (Base.or_int)(%8, %14)::Bool
│       └───       goto #7
│       4 ── %17 = (isa)(%3, Int64)::Bool
│       └───       goto #6 if not %17
│       5 ── %19 = π (%3, Int64)
││╻     <  │    %20 = (Base.slt_int)(1, %19)::Bool
│       └───       goto #7
│       

In [44]:
function sumlog(xs)
    s = zero(xs[1])
    
    for i=1:length(xs)
        s = s + logint2(xs[i]) # If type of xs[i] is known, result of logint2 will be known
    end
    s
end

sumlog (generic function with 1 method)

In [57]:
@code_typed optimize=false sumlog(xs)

CodeInfo(
│2 1 ─ %1  = (Base.getindex)(xs, 1)::Float64
│  │         (s = (Main.zero)(%1))::Const(0.0, false)
│4 │   %3  = (Main.length)(xs)::Int64
│  │   %4  = (1:%3)::UnitRange{Int64}
│  │         (#temp# = (Base.iterate)(%4))::Union{Nothing, Tuple{Int64,Int64}}
│  │   %6  = (#temp# === nothing)::Bool
│  │   %7  = (Base.not_int)(%6)::Bool
│  └──       goto #4 if not %7
│  2 ┄ %9  = #temp#::Tuple{Int64,Int64}::Tuple{Int64,Int64}
│  │         (i = (Core.getfield)(%9, 1))::Int64
│  │   %11 = (Core.getfield)(%9, 2)::Int64
│5 │   %12 = s::Float64
│  │   %13 = (Base.getindex)(xs, i)::Float64
│  │   %14 = (Main.logint2)(%13)::Int64
│  │         (s = %12 + %14)::Float64
│  │         (#temp# = (Base.iterate)(%4, %11))::Union{Nothing, Tuple{Int64,Int64}}
│  │   %17 = (#temp# === nothing)::Bool
│  │   %18 = (Base.not_int)(%17)::Bool
│  └──       goto #4 if not %18
│  3 ─       goto #2
│7 4 ─       return s
) => Float64

In [65]:
@time logint2.([i for i in 1:2^20]);

  0.097134 seconds (55.88 k allocations: 18.758 MiB, 1.91% gc time)


# Summary

**Uncertainty** is the root cause of slowness.

It can come in many forms!

1. Globals -- uncertainty about type after compilation
2. Type instability -- uncertainty about the input or output type
3. Not fixed length -- arrays or strings don't have a fixed size
3. Mutability -- uncertainty that someone else may mutate the object
4. Boxing -- uncertainty about type or, size of values

Use `@time` and pay attention to allocations -- uncertainty leads to boxing which leads to GC overhead, so you will see it!

## References

- Performance tips from the manual https://docs.julialang.org/en/v0.6.1/manual/performance-tips/
- Engineering Julia for Speed - Lionel Zoubritzky
 https://www.youtube.com/watch?v=XWIZ_dCO6X8
- Information overload: tools for making program analysis and debugging manageable - Jameson Nash JuliaCon 2018